In [0]:
from planet4 import region_data, io
from pathlib import Path

In [0]:
root = io.analysis_folder() / 'inca_s23_0.5cut_applied/'

In [0]:
fan_fnames = list(root.glob("*_fans_latlons.csv"))

In [0]:
blotch_fnames = list(root.glob("*_blotches_latlons.csv"))

In [0]:
from planet4 import markings

In [0]:
def get_marking(line, cut=0.8):
    data = []
    for marking in ['fan_','blotch_']:
        m = line[line.index.str.startswith(marking)]
        data.append(m.rename_axis(lambda x: x[x.index('_')+1:]))
    fnotch = markings.Fnotch(line.fnotch_value, data[0], data[1])
    return fnotch.get_marking(cut)

In [0]:
def get_final_markings_counts(img_name, cut=0.5):
    # writing in dictionary here b/c later I convert it to pd.DataFrame
    # for which a dictionary is a natural input format
    d = {}
    d['image_name'] = img_name
    blotch_fname = root / '{}_blotches_latlons.csv'.format(img_name)
    d['n_blotches'] = len(pd.read_csv(str(blotch_fname)))
    fan_fname = root / '{}_fans_latlons.csv'.format(img_name)
    d['n_fans'] = len(pd.read_csv(str(fan_fname)))
    
    return d

In [0]:
get_final_markings_counts('ESP_020115_0985')

In [0]:
results = []
from IPython.display import display
from ipywidgets import IntProgress
t = IntProgress(min=0, max=len(region_data.Inca.season2)-1)
display(t)
for i, img_name in enumerate(region_data.Inca.season2):
    t.value = i
    try:
        results.append(get_final_markings_counts(img_name))
    except OSError:
        continue
season2 = pd.DataFrame(results).sort_values(by='image_name')

In [0]:
results = []
from IPython.display import display
from ipywidgets import IntProgress
t = IntProgress(min=0, max=len(region_data.Inca.season3)-1)
display(t)
for i, img_name in enumerate(region_data.Inca.season3):
    t.value = i
    try:
        results.append(get_final_markings_counts(img_name))
    except OSError:
        continue
season3 = pd.DataFrame(results).sort_values(by='image_name')

In [0]:
season2.info()

In [0]:
%matplotlib inline

In [0]:
metadata = pd.read_csv("/Users/klay6683/Dropbox/data/planet4/P4_10-18-15_H_lat_lng.csv")

In [0]:
metadata

In [0]:
from hirise.hirise_tools import get_rdr_label, labels_root

In [0]:
import pvl
def get_nlines_from_label(labelfname):
    module = pvl.load(str(labelfname))
    return module['UNCOMPRESSED_FILE']['IMAGE']['LINE_SAMPLES']

In [0]:
p = labels_root()

In [0]:
metadata['labelpath'] = metadata.HiRISE_image.map(lambda x: p / (x + '_RED.LBL'))

In [0]:
metadata['nsamples'] = metadata.labelpath.map(get_nlines_from_label)

In [0]:
metadata.set_index('HiRISE_image', inplace=True)

In [0]:
season2.set_index('image_name', inplace=True)
season3.set_index('image_name', inplace=True)

In [0]:
season2 = season2.join(metadata['solar_longitude binning nsamples'.split()])

In [0]:
season3 = season3.join(metadata['solar_longitude binning nsamples'.split()])

In [0]:
season2.set_index('solar_longitude', inplace=True)

In [0]:
season3.set_index('solar_longitude', inplace=True)

In [0]:
season2['both'] = season2.n_blotches + season2.n_fans

In [0]:
season3['both'] = season3.n_blotches + season3.n_fans

In [0]:
season2['scaled'] = season2.both / season2.nsamples / season2.binning

In [0]:
season3['scaled'] = season3.both / season3.nsamples / season3.binning

In [0]:
%matplotlib notebook
import seaborn as sns
sns.set()

In [0]:
season2.scaled.plot(style='*', ms=14, 
                    xlim=(season3.index.min(), season3.index.max()), label='season2')
season3.scaled.plot(style='*', color='red', label='season3', ms=14)
plt.legend(loc='best')
plt.ylabel('Scaled prevalence of markings')
plt.title("Number of markings in Inca City region,scaled for binning and image size.")
plt.savefig('/Users/klay6683/Desktop/inca_s23.pdf')

In [0]:
season2

In [0]:
season3

In [0]:
season3.scaled.plot()